# __{튜토리얼 제목}__

- 튜토리얼 난이도: {★☆☆☆☆}
- 읽는데 걸리는 시간: {0}분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_ml/{폴더 이름}/{파일 이름}.ipynb
- 참고 문서: [데이터 세트](다운로드 가능한 링크), [참고 문서](논문 혹은 개념 설명 글)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">OOO 이해하기</h4>
    <p>해당 모델 작업에 대한 설명을 적습니다. (예시) 분류 작업은 목푯값(Target)이 속한 범주(Category 또는 Class)를 예측하기 위해 사용하는 <a href="https://ko.wikipedia.org/wiki/%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5">머신러닝(기계학습/Machine Learning)</a>의 한 형태입니다. 예를 들어, 남성 또는 여성을 분류하는 이진 분류와 동물의 종(개, 고양이, 토끼 등)을 예측하는 다중 분류 모두 분류 작업에 포함됩니다.</p>
</div>

추가적인 설명이 필요한 경우 여기에 적습니다.

(예시) 기업의 특정 마케팅 프로모션에 대해 잠재고객이 긍정적인 반응을 보일 것인지 아닌지를 예측하기 위해서는 고객의 [CRM(Customer Relationship Management)](https://ko.wikipedia.org/wiki/%EA%B3%A0%EA%B0%9D_%EA%B4%80%EA%B3%84_%EA%B4%80%EB%A6%AC) 데이터(인구 통계학 정보, 고객의 행동/검색 데이터 등)를 이용할 수 있습니다. 이 경우 CRM 데이터에서 표현되는 <a href="https://ko.wikipedia.org/wiki/%ED%8A%B9%EC%A7%95_(%EA%B8%B0%EA%B3%84_%ED%95%99%EC%8A%B5)">특성(Feature)</a>이 입력 데이터로 사용되며, 예측하고자 하는 값인 목푯값은 프로모션에 대한 대상 고객의 반응이 긍정(1 또는 True) 또는 부정(0 또는 False)일지 여부입니다. 이러한 분류 모델을 이용해서, 마케팅에 노출되지 않은 고객들의 프로모션에 대한 반응을 미리 예측하고 적절한 고객에게 마케팅을 노출함으로써 마케팅 효율을 지속적으로 높일 수 있습니다. 

__아래는 ThanoSQL OOO 모델의 활용 및 예시입니다.__

- ThanoSQL 활용방안 및 예시를 2~3개 적습니다.
- (예시) 온라인 플랫폼 내에서 사용자의 [세그먼트](https://ko.wikipedia.org/wiki/%EC%8B%9C%EC%9E%A5%EC%84%B8%EB%B6%84%ED%99%94)를 예측할 수 있습니다. 대부분의 서비스 사용자들은 서로 다른 특성을 가지고 다양한 행동방식(User Behaviour)과 니즈(Needs)를 가지고 있습니다. 분류 예측 모델은 서비스 사용자의 특성을 이용하여 세분화된 집단을 식별하고 그들에게 맞춤화된 전략 수립을 가능하게 합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 해당 튜토리얼에 대한 설명을 적습니다.</p>
</div>

<div class="admonition tip">
    <h4 class="admonition-title">(Optional)데이터 세트 설명</h4>
    <p>데이터 세트 설명이 필요하면 추가로 작성합니다. 필요 없을 시 생략 가능</p>
</div>
    
추가적인 설명이 필요한 경우 적어줍니다. (예시) 이제부터 ThanoSQL을 사용하여 간단하게 타이타닉에서 살아남을 수 있는 승객을 예측하는 분류 모델을 만들어 봅니다.

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/how_to_use_ThanoSQL/#5-thanosql)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

(또는) (pre-built 모델이 있는 경우)

## __0. 데이터 세트 및 모델 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/how_to_use_ThanoSQL/#5-thanosql)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET {데이터 세트}
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY {테이블 이름}
OPTIONS (overwrite=True)
FROM 'csv 파일 위치'

Success


In [4]:
(예시)
%%thanosql
COPY titanic_test
OPTIONS (overwrite=True)
FROM 'thanosql-dataset/titanic_data/titanic_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 테이블이 데이터베이스 상에 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 테이블은 새로운 테이블로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

#### (pre-built 모델이 있는 경우)

### __모델 준비__

In [4]:
%%thanosql
GET THANOSQL MODEL {모델 이름}
OPTIONS (
    model_name='tutorial_모델이름',
    overwrite=True
    )

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL MODEL</strong>" 쿼리 구문을 사용하여 원하는 모델을 워크스페이스 및 데이터베이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL MODEL</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>“model_name”: 저장할 모델의 이름 (str, optional)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

OOO 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __{테이블 이름}__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

(또는) (BUILD를 지원하지 않을 경우)

## __1. 데이터 세트 확인__

본 튜토리얼을 진행하기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __{테이블 이름}__ 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM {테이블 이름}
LIMIT 5

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>{테이블 이름}</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>{칼럼명}: 칼럼 설명</li>
        <li>(예시) survived: 탑승승객 생존 여부</li>
    </ul>
</div>

#### (pre-built 모델이 있는 경우)

## __2. 사전 학습된 모델을 사용하여 OOO 예측__

다음 쿼리 구문을 실행하여 __{pre-built 모델 이름}__ 모델을 사용하여 결과를 예측합니다.

In [ ]:
%%thanosql
PREDICT USING {pre-built 모델 이름}
OPTIONS (
    (예시) image_col='image_path',
    (예시) result_col='predict_result'
    )
AS
SELECT *
FROM {테이블 이름}

## __2. OOO 모델 생성__

이전 단계에서 확인한 __{테이블 이름}__ 테이블을 사용하여 OOO 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>{모델 이름}</strong>이라는 이름의 모델을 만듭니다.\
(쿼리 실행 시 예상 소요 시간: {0} min)

In [6]:
%%thanosql
BUILD MODEL {모델 이름}
USING {베이스 모델 이름}
OPTIONS (
    필요한_옵션=추가,
    overwrite=True
    )
AS
SELECT *
FROM {테이블 이름}

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>{모델 이름}</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>{베이스 모델 이름}</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>(예시) "target_col": 분류 모델의 목푯값이 담겨 있는 컬럼명 (str, default: 'target')</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 유무 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __3. (생성된) 모델을 사용하여 OOO 예측__

이전 단계에서 (생성한) OOO 모델을 사용해 OOO을 예측해 봅니다. 테스트용 데이터 세트(학습에 이용되지 않은 데이터 테이블, <strong>{테이블 이름}</strong>)를 사용합니다.

In [8]:
%%thanosql
PREDICT USING {모델 이름}
OPTIONS (
    (예시) image_col='image_path',
    (예시) result_col='predict_result'
    (예시) table_name='{테이블 이름}'
    )
AS
SELECT *
FROM {테이블 이름}

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,predict_result
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,None,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,None,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,None,S,0


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>PREDICT USING</strong>" 쿼리 구문을 사용하여 이전 단계에서 만든 <strong>{모델 이름}</strong> 모델을 예측에 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 예측에 사용할 옵션을 지정합니다.
        <ul>
            <li>(예시) "image_col": 예측에 사용할 이미지의 경로가 기록되어 있는 컬럼의 이름 (str, default: 'image_path')</li>
            <li>(예시) "result_col": 데이터 테이블에서 예측 결과를 담을 컬럼 이름 (str, optional, default: 'predict_result')</li>
            <li>(예시) "table_name": ThanoSQL 워크스페이스 데이터베이스 내에 저장될 테이블 이름. 기존에 사용한 테이블 이름으로 지정할 경우, 기존 테이블은 'predict_result' 컬럼을 추가한 테이블로 대체. 지정하지 않을 시 테이블을 저장하지 않음 (str, optional)</li>
        </ul>
        </li>
    </ul>
</div>

## __4. 튜토리얼을 마치며__

이번 튜토리얼에서는 {데이터 세트} 데이터 세트를 사용하여 OOO 모델을 만들어 보았습니다. (뒤에 내용 추가)

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>OOO 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>